## sequence-to-sequence learning in keras

In [1]:
from __future__ import print_function

from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense, Bidirectional, Concatenate
from IPython.display import SVG

import numpy as np

Using TensorFlow backend.


In [2]:
batch_size = 64  # Batch size for training
epochs = 100  # Number of epochs to train for
latent_dim = 256  # Latent dimensionality of the encoding space.

input_data_path = 'data/aion_train_v.txt'
target_data_path = "data/aion_train_t.txt"

input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(input_data_path, "r", encoding="utf-8") as f:
    reader = f.readlines()
    for i in reader:
        input_texts.append(i)
        for char in i:
            if char not in input_characters:
                input_characters.add(char)
            
with open(target_data_path, "r", encoding="utf-8") as f:
    reader = f.readlines()
    for i in reader:
        # '\t'  = start sequence
        # '\n' = end sequence
        target_text = '\t' + i
        target_texts.append(target_text)
        for char in target_text:
            if char not in target_characters:
                target_characters.add(char)

In [3]:
input_characters = sorted(list(input_characters)) ## character를 순서대로 정리 ㄱ~ㅎ
target_characters = sorted(list(target_characters))  ## character를 순서대로 정리 a~z ㄱ~ㅎ
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])  ## input texts에 있는 문장 길이를 순서대로 다른 리스트에 등록하고 최대 찾기
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [4]:
print(input_characters)

['\n', ' ', '.', '가', '각', '갈', '감', '강', '거', '검', '게', '겠', '견', '결', '계', '고', '곰', '관', '광', '귄', '그', '근', '금', '기', '까', '깡', '끼', '나', '냥', '네', '넬', '능', '니', '다', '단', '달', '대', '더', '데', '도', '동', '둠', '드', '디', '라', '랄', '랑', '랙', '량', '레', '렌', '려', '력', '렴', '로', '룡', '루', '룬', '룽', '르', '를', '리', '린', '마', '막', '만', '망', '맹', '메', '명', '모', '몽', '무', '묵', '민', '바', '반', '백', '벌', '베', '벨', '변', '보', '복', '불', '브', '블', '비', '빅', '빛', '빨', '사', '상', '새', '생', '샤', '서', '설', '성', '세', '셀', '소', '수', '쉬', '슈', '스', '승', '시', '신', '실', '심', '십', '아', '악', '안', '야', '양', '어', '에', '여', '역', '영', '예', '옐', '오', '와', '요', '용', '우', '웨', '유', '은', '을', '의', '이', '인', '일', '자', '작', '적', '전', '점', '정', '제', '좀', '주', '줘', '지', '진', '집', '징', '채', '챈', '천', '체', '초', '최', '추', '축', '치', '침', '카', '칸', '켈', '쿠', '퀘', '큐', '크', '키', '타', '탑', '테', '토', '톤', '통', '투', '트', '티', '팅', '파', '팬', '퍼', '펭', '포', '표', '프', '플', '픽', '핑', '하', '한', '합', '해', '험', '헤', '혁', '현', '혜', '호', '황'

In [5]:
print(target_characters)

['\t', '\n', ' ', '(', ')', '.', 'a', 'b', 'c', 'e', 'f', 'h', 'i', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', '가', '각', '갈', '감', '강', '검', '견', '결', '계', '고', '곰', '관', '광', '귄', '그', '근', '금', '기', '깡', '끼', '나', '냥', '네', '넬', '능', '니', '다', '단', '달', '대', '더', '데', '도', '동', '둠', '드', '디', '라', '랄', '랑', '랙', '량', '레', '렌', '력', '로', '룡', '루', '룬', '룽', '르', '리', '린', '마', '막', '만', '망', '맹', '메', '명', '모', '몽', '묵', '민', '바', '반', '백', '벌', '베', '벨', '보', '복', '불', '브', '블', '비', '빛', '사', '상', '새', '생', '샤', '성', '세', '셀', '수', '쉬', '슈', '스', '승', '시', '신', '실', '심', '아', '악', '안', '야', '양', '어', '에', '여', '역', '영', '예', '옐', '오', '와', '요', '용', '우', '웨', '유', '은', '을', '의', '이', '인', '일', '작', '적', '전', '정', '제', '지', '진', '집', '징', '채', '챈', '천', '체', '초', '최', '추', '축', '치', '침', '카', '칸', '켈', '쿠', '크', '키', '타', '탑', '테', '토', '톤', '통', '투', '트', '티', '파', '팬', '퍼', '펭', '포', '표', '프', '플', '픽', '핑', '하', '한', '해', '험', '헤', '혁', '현', '혜', '호', '황']


In [6]:
print('Number of samples:', len(input_texts))
print('Number of unique input characters:', num_encoder_tokens)
print('Number of unique output characters:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 15182
Number of unique input characters: 200
Number of unique output characters: 196
Max sequence length for inputs: 27
Max sequence length for outputs: 23


In [7]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [8]:
## one hot vector 생성 - [0,1,...0,...0,0]
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [9]:
encoder_inputs = Input(shape=(None, num_encoder_tokens)) ##[문장길이, one-hot vector길이]
encoder = Bidirectional(LSTM(latent_dim, return_state=True),merge_mode="sum") ## latent_dim == size
# encoder = LSTM(latent_dim, return_state=True)## latent_dim == size
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(encoder_inputs)
# encoder_outputs, state_h, state_c = encoder(encoder_inputs)

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
encoder_states = [state_h, state_c]

In [10]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim*2, return_sequences=True, return_state=True) ## return_sequences -> output이 다음 state의 input으로 들어감
# decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

## 모델 저장시 Warning 발생하지만 무시. // keras 버그 인듯

In [11]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('model_save/aion_slu_lstm.h5')

Train on 12145 samples, validate on 3037 samples
Epoch 1/100
12145/12145 [==============================] - 14s 1ms/step - loss: 1.3905 - val_loss: 0.7254
Epoch 2/100
12145/12145 [==============================] - 13s 1ms/step - loss: 0.4110 - val_loss: 0.1840
Epoch 3/100
12145/12145 [==============================] - 13s 1ms/step - loss: 0.1207 - val_loss: 0.0636
Epoch 4/100
12145/12145 [==============================] - 13s 1ms/step - loss: 0.0583 - val_loss: 0.0032
Epoch 5/100
12145/12145 [==============================] - 13s 1ms/step - loss: 0.0067 - val_loss: 1.4646e-04
Epoch 6/100
12145/12145 [==============================] - 13s 1ms/step - loss: 0.0076 - val_loss: 7.5323e-05
Epoch 7/100
12145/12145 [==============================] - 13s 1ms/step - loss: 0.0029 - val_loss: 8.9756e-06
Epoch 8/100
12145/12145 [==============================] - 13s 1ms/step - loss: 0.0033 - val_loss: 2.8671e-06
Epoch 9/100
12145/12145 [==============================] - 13s 1ms/step - loss: 0.0029 

12145/12145 [==============================] - 13s 1ms/step - loss: 8.3072e-08 - val_loss: 8.2988e-08
Epoch 73/100
12145/12145 [==============================] - 13s 1ms/step - loss: 8.3045e-08 - val_loss: 8.2948e-08
Epoch 74/100
12145/12145 [==============================] - 13s 1ms/step - loss: 8.3004e-08 - val_loss: 8.2903e-08
Epoch 75/100
12145/12145 [==============================] - 13s 1ms/step - loss: 8.2975e-08 - val_loss: 8.2883e-08
Epoch 76/100
12145/12145 [==============================] - 13s 1ms/step - loss: 8.2955e-08 - val_loss: 8.2863e-08
Epoch 77/100
12145/12145 [==============================] - 13s 1ms/step - loss: 8.2946e-08 - val_loss: 8.2851e-08
Epoch 78/100
12145/12145 [==============================] - 13s 1ms/step - loss: 8.2939e-08 - val_loss: 8.2842e-08
Epoch 79/100
12145/12145 [==============================] - 13s 1ms/step - loss: 8.2930e-08 - val_loss: 8.2840e-08
Epoch 80/100
12145/12145 [==============================] - 13s 1ms/step - loss: 8.2929e-08 -

/home/dongjin/anaconda3/envs/my_1.8/lib/python3.5/site-packages/keras/engine/network.py:872: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'concatenate_1/concat:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'concatenate_2/concat:0' shape=(?, 512) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [43]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim*2,))
decoder_state_input_c = Input(shape=(latent_dim*2,))
#decoder_state_input_h = Input(shape=(latent_dim,))
#decoder_state_input_c = Input(shape=(latent_dim,))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs,
                     [decoder_outputs] + decoder_states)

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [44]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == "\n" or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

## Training Data

In [45]:
for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: 감정 표현 동작 하자.

Decoded sentence: motion(감정 표현).

-
Input sentence: 에레슈키갈의 사도 변신을 시작 해줘.

Decoded sentence: transform(에레슈키갈의 사도).

-
Input sentence: 정신 큐빅을 시작 하소서.

Decoded sentence: cubic(정신).

-
Input sentence: 황금 깡통 변신을 하게나.

Decoded sentence: transform(황금 깡통).

-
Input sentence: 최근 퀘스트 빨리 하지.

Decoded sentence: quest(최근).

-
Input sentence: 카이시넬 변신을 빨리 합시다.

Decoded sentence: transform(카이시넬).

-
Input sentence: 감정 표현 동작 하여라.

Decoded sentence: motion(감정 표현).

-
Input sentence: 루나 파견대 변신을 빨리 하게.

Decoded sentence: transform(루나 파견대).

-
Input sentence: 광대 변신을 빨리 하여라.

Decoded sentence: transform(광대).

-
Input sentence: 징표 동작 지금 좀.

Decoded sentence: motion(징표).



## Test Data

In [46]:
test_input_texts = []

with open("data/aion_test_v.txt", "r", encoding='utf=8') as f:
    reader =f.readlines()
    for i in reader:
        test_input_texts.append(i)

test_encoder_input_data = np.zeros((len(test_input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')

for i, test_input_text in enumerate(test_input_texts):
    for t, char in enumerate(test_input_text):
        test_encoder_input_data[i, t, input_token_index[char]] = 1.
        
for seq_index in range(28, 30):
    input_seq = test_encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', test_input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: 레파르 혁명단 변신 시작 하렴.

Decoded sentence: transform(레파르 혁명단).

-
Input sentence: 레드 펭귄 변신을 지금 하십시오.

Decoded sentence: transform(레드 펭귄).

